In [1]:
'''

https://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/issues/594

Homologação: http://acs-assist-sem-rag.nia.hm.bb.com.br/nia_semantica_acs/docs

Desenvolvimento: http://acs-assist-sem-rag.nia.desenv.bb.com.br/nia_semantica_acs/docs

nesse primeiro exemplo vamos trabalhar com agentes sem rag para maior liberdade e entendimento de como funciona agentes

documentação:

https://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/wikis/home


Base de conhecimento RAG

https://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?sw=auth#declara%c3%a7%c3%a3o-de-assistente
https://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?#indexa%C3%A7%C3%A3o-assistente

'''

'\n\nhttps://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/issues/594\n\nHomologação: http://acs-assist-sem-rag.nia.hm.bb.com.br/nia_semantica_acs/docs\n\nDesenvolvimento: http://acs-assist-sem-rag.nia.desenv.bb.com.br/nia_semantica_acs/docs\n\nnesse primeiro exemplo vamos trabalhar com agentes sem rag para maior liberdade e entendimento de como funciona agentes\n\ndocumentação:\n\nhttps://fontes.intranet.bb.com.br/nia/publico/llm-atendimento/-/wikis/home\n\n\nBase de conhecimento RAG\n\nhttps://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?sw=auth#declara%c3%a7%c3%a3o-de-assistente\nhttps://banco365.sharepoint.com/sites/PlataformaConversacional2/SitePages/Pipeline-de-Assistentes-Inteligentes.aspx?#indexa%C3%A7%C3%A3o-assistente\n\n'

RAG: Retrieval-Augmented Generation (Geração Aumentada por Recuperação). Essa técnica combina a geração de texto com a recuperação de informações de fontes externas para melhorar a precisão e a relevância das respostas geradas pelos modelos de linguagem.

Exemplo:

Recuperação de Informações: Quando o modelo recebe uma consulta, ele primeiro busca informações relevantes em uma base de dados externa ou em documentos específicos.
Geração de Texto: Em seguida, o modelo usa essas informações recuperadas para gerar uma resposta mais precisa e informada.

Como nesse agente cadastramos sem RAG cabe ao deve passar as informações relevantes

exemplo pratico no BB:

Aqui está um relatório de auditoria especial. Quero que você identifique e descreva o modus operandi utilizado nas irregularidades encontradas.

**Conceito de Modus Operandi**:
Modus operandi é um termo usado para descrever os métodos e processos utilizados por uma pessoa ou grupo para cometer uma fraude ou irregularidade.

**Exemplos de Modus Operandi**:
1. Uso de documentos falsificados para abrir contas bancárias.
2. Manipulação de registros financeiros para ocultar desfalques.
3. Uso de identidades falsas para realizar transações fraudulentas.

**Relatório de Auditoria Especial**:
[Insira o texto do relatório aqui]

Com base nas informações acima, identifique e descreva o modus operandi utilizado nas irregularidades encontradas no relatório.


In [2]:
# controlar o máximo de envio de informação
import tiktoken

In [3]:
import pandas as pd
import os
import sys
import numpy 
import json
import time
import requests
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, types
from IPython.display import display, JSON

In [4]:
# Carregue o encoding para o modelo GPT-3.5-turbo (fica proximo do modelo atual)
encoding = tiktoken.encoding_for_model("gpt-4")

In [5]:
from dotenv import load_dotenv

In [6]:
dotenv_path = '/code/notebooks/taxonomia/.env'
load_dotenv(dotenv_path)
usr = os.getenv('USR_PG_SALA_PLANEJAI')
pw = os.getenv('PWD_PG_SALA_PLANEJAI')

In [7]:
# Caminho para o diretório onde o módulo conexoes.py está localizado
caminho_para_conexoes = '/code/notebooks/taxonomia'

# Adicionando o caminho ao sys.path
if caminho_para_conexoes not in sys.path:
    sys.path.append(caminho_para_conexoes)

# Importando o módulo conexoes
from conexoes import *

In [8]:
def assistenteExemplo(row):
    
    # aqui é um exemplo de orientação que estou passando para api
    # depois em produção deve deixar organizado em um modulo a parte, como esse agente não tem RAG (base de conhecimento), cabe ao dev passar as informações relevantes
    # pode criar agentes com RAG - base de conhecimento, combinar agentes entre outras opções
    
    orientacao_prompt = '''Você é um assistente de auditoria especializado em vincular processos corporativos de auditoria. Sua tarefa é classificar processos de apuração de irregularidades em categorias específicas de risco corporativo e retornar o resultado no formato JSON. As categorias de risco são:
    1.Risco de Estratégia; 2.Risco de Crédito; 3.Risco Atuarial; 4.Risco de IRRBB; 5.Risco de Mercado; 6.Risco de Liquidez; 7.Risco de Contágio; 18.Risco Social, Ambiental e Climático; 45.Risco Cibernético; 48.Risco de Conformidade; 49.Risco de Terceiros; 50.Risco Legal; 776.Risco de Conduta; 777.Risco de Modelo; 778.Risco de Segurança; 779.Risco de TI
    Classifique a apuração de irregularidades em uma ou mais das categorias de risco corporativo listadas acima. Para cada categoria, indique "Sim" ou "Não". Além disso, crie uma categoria chamada regra_classificacao que descreve qual a lógica que você usou para classificar.
    Importante:
    1) A base de conhecimento possui "Definição Principal" e "Definição Secundária", delimitadas entre [] sendo que a classificação de risco precisa observar obrigatóriamente a "Definição Principal" sendo que a "Definição Secundária" está contida na "Definição Principal". 
    Exemplo: [Definição Principal:Possibilidade de perdas decorrentes do relacionamento do Banco com terceiros.]
    2) A classificação de riscos a ser realizada precisa dar maior peso à classificação mencionada no item 1).
  
# Formato Saída
{
    "regra_classificacao": "foi classificada como sim Risco Operacional, Risco Financeiro porque houve uma falha no sistema de processamento, levando a um prejuízo financeiro significativo.",
    "risco_estrategia": "Não",
    "risco_credito": "Não",
    "risco_atuarial": "Sim",
    "risco_IRRBB": "Não",
    "risco_mercado": "Não",
    "risco_liquidez": "Não",
    "risco_contagio": "Não",
    "risco_social_ambiental_climatico": "Não",
    "risco_cibernetico": "Não",
    "risco_conformidade": "Não",
    "risco_terceiros": "Não",
    "risco_legal": "Não",
    "risco_conduta": "Não",
    "risco_modelo": "Não",
    "risco_seguranca": "Não",
    "risco_ti": "Não"
} 

OBS.: Envie o JSON em formato puro sem vestígios de markdown, remove a marcação abaixo:
[REMOVER]:
```json
```
    Aqui está os processos de apuração de irregularidades:    
    '''
    
    avaliacao_nota_tecnica = f"irregularidades: {row['tx_gpt']} "
    
    prompt = orientacao_prompt + avaliacao_nota_tecnica + 'segue lista da base de conhecimento'
    
    
    url = 'http://acs-assist-sem-rag.nia.hm.bb.com.br/acs/llms/agent'
    resp = requests.post(url, data=json.dumps({
        "data": {
            "input": prompt,
            "intents": [{}],
            "entities": [{}],
            "context": {
                "config": {
                    "top_p": 0.95,
                    "max_tokens": 4000,
                    "temperature": 0.2
                }
            }
        }
    }), headers={'Content-type': 'application/json', 'accept': 'application/json'})

    # Convertendo a resposta em um dicionário
    resp_dict = json.loads(resp.text)

    # Extraindo o status
    status = resp_dict.get('status')
    print('status=======> ', status)

    # Extraindo o resultado da resposta
    texto_resposta = resp_dict["data"]["context"]["prompt_response"]["acs_llm_prompt_execution"]["result"]
    
    print('texto_resposta: ', texto_resposta)
    
    return texto_resposta


In [9]:
# cria conexao
conn, engine = postgres(db="sala_agil", usr=usr, pw=pw)
pd.set_option('display.max_colwidth', None)

In [10]:
# Consultar dados do PostgreSQL incluindo as colunas extras
query_postgres = f"""SELECT *
        FROM riscometro.dmd_ae_gpt  limit 5;
    """
df_postgres = pd.read_sql(query_postgres, conn)

/tmp/ipykernel_205/2071514230.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_postgres = pd.read_sql(query_postgres, conn)


In [11]:
def contar_tokens(texto):
    tokens = encoding.encode(texto)
    return len(tokens)

In [12]:
df_postgres['quantidade_tokens'] = df_postgres['tx_gpt'].apply(contar_tokens)

In [13]:
# controle com o maximo de tokens para envio, conforme serviço cadastrado no nia
df_postgres = df_postgres.query('quantidade_tokens <= 10000')
# Resetando o índice
df_postgres.reset_index(drop=True, inplace=True)

In [14]:
df_postgres.head(5)

indice  ano_dmd                                          nm_irr  \
0     1.0   2023.0           Abertura de conta com documento falso   
1     2.0   2023.0           Abertura de conta com documento falso   
2     3.0   2023.0    Abertura de conta sem autorização do titular   
3     4.0   2023.0    Abertura de conta sem autorização do titular   
4     5.0   2023.0  Abertura de conta sem documentação obrigatória   

                  nm_enq  \
0         Comportamental   
1                 Fraude   
2  Desvio Ético Negocial   
3                 Fraude   
4    Atividade imprópria   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Abaixo tem um exemplo com iteração usando FOR para entender o que acontece por debaixo dos panos. No entanto, para produção, tem que seguir as Boas Práticas:

Limite de Tentativas: Evita loops infinitos.
Mensagens de Log: Facilita o monitoramento.
Recarregar Dados: Processa apenas dados não processados.
Pausas: Evita sobrecarga.
Tratamento de Exceções: Lida com erros sem interromper o script.
Atualização Condicional: Garante dados válidos.
Reset de Índice: Mantém a consistência dos dados.

In [15]:
# exemplo de chamada, que reaproveito minha tabela com informações adicionar
# fiz um exemplo em for para entender o que acontece por debaixo dos pannos, mas para produção o ideal é fazer com while, verificar 

df_local = pd.DataFrame()  # DataFrame para armazenar os resultados

for index, row in df_postgres.iterrows():
    if index > 0 and index % 10 == 0:
        print("Aguardando 10 segundos...")
        time.sleep(10)  # Espera 10 segundos a cada 10 iterações

    try:
        proposta = assistenteExemplo(row)
        
        # Converte a proposta de string JSON para dicionário, se necessário, posso converter no retorno da função api ou aqui, dev decide
        if isinstance(proposta, str):
            proposta = json.loads(proposta)
        
        # Atualiza o dicionário de resultados com a proposta
        resultado = row.to_dict()
        resultado.update(proposta)
        
        # Fazendo o append do resultado no DataFrame local
        df_local = pd.concat([df_local, pd.DataFrame([resultado])], ignore_index=True)

    except Exception as e:
        print(f"Erro {row['indice']}: {e}")
        # Não salva a linha problemática para que seja processada novamente

# salvei local apenas para ver o resultado, em porodução salva no postgres, por exemplo, e agenda no cron so servidor
df_local.to_csv('resultados_locais.csv', index=False)


status=======>  200
texto_resposta:  {
    "regra_classificacao": "As irregularidades foram classificadas com base na 'Definição Principal' de cada categoria de risco, considerando a natureza das informações e atividades descritas nas irregularidades.",
    "risco_estrategia": "Não",
    "risco_credito": "Sim",
    "risco_atuarial": "Não",
    "risco_IRRBB": "Não",
    "risco_mercado": "Não",
    "risco_liquidez": "Não",
    "risco_contagio": "Não",
    "risco_social_ambiental_climatico": "Não",
    "risco_cibernetico": "Sim",
    "risco_conformidade": "Sim",
    "risco_terceiros": "Sim",
    "risco_legal": "Sim",
    "risco_conduta": "Sim",
    "risco_modelo": "Não",
    "risco_seguranca": "Sim",
    "risco_ti": "Sim"
}
status=======>  200
texto_resposta:  ```json
{
    "regra_classificacao": "As irregularidades foram classificadas com base na natureza das atividades descritas, que envolvem principalmente questões de documentação falsa, abertura de contas e concessão de crédito sem as

In [16]:
df_local

,indice,ano_dmd,nm_irr,nm_enq,tx_gpt,qtd_caract,quantidade_tokens,regra_classificacao,risco_estrategia,risco_credito,...,risco_contagio,risco_social_ambiental_climatico,risco_cibernetico,risco_conformidade,risco_terceiros,risco_legal,risco_conduta,risco_modelo,risco_seguranca,risco_ti
0,1.0,2023.0,Abertura de conta com documento falso,Comportamental,1 Repasse à terceiro não autorizado de informações de clientes protegidas pelo sigilo bancário Arquivo 05 páginas 38 a 46 br 2 Registros de faturamento renda pessoa jurídica embasados em documentos não validados e ou com informações inconsistentes br 3 Contratação de operações de crédito para pessoas jurídicas cujos registros de faturamento foram embasados em documentos não validados ou com informações inconsistentes br 4 Inclusão de comprovantes de endereços nos dossiês eletrônicos de clientes DEL pessoa física com base em documentos não fidedignos br 5 Abertura de contas correntes pessoa física e ou cadastramento de equipamentos números de telefone de forma irregular br,718.0,195,"As irregularidades foram classificadas com base na 'Definição Principal' de cada categoria de risco, considerando a natureza das informações e atividades descritas nas irregularidades.",Não,Sim,...,Não,Não,Sim,Sim,Sim,Sim,Sim,Não,Sim,Sim
1,3.0,2023.0,Abertura de conta sem autorização do titular,Desvio Ético Negocial,Procedimentos de abertura e conformidade de contas correntes sem prévia e formal autorização dos respectivos clientes e com vistas a propiciar o cumprimento de meta estabelecida para a dependência,197.0,46,"A apuração foi classificada como Risco de Conduta porque envolve procedimentos inadequados na abertura de contas correntes sem autorização dos clientes, o que está diretamente relacionado com a conduta dos funcionários do banco e pode levar a perdas financeiras e de reputação.",Não,Não,...,Não,Não,Não,Sim,Não,Sim,Sim,Não,Não,Não
